NLP on Letterboxd df
- Check word frequency in several columns -> 'tagline', 'summary', 'genres', 'themes', 'events'
- Based on correlations, rate from 0 to 5
- Tie back to whether a title has warnings or not
- Figure how it impacts sentiment on movies
- Sentiment-o-meter

The Data
- `cleaned_films` contains 4203 titles and 13 columns.
- There are no Null values.

| **Column Name**        | **Data Type**   | **Description**                                                                 |
|------------------------|-----------------|----------------------------------------------------------------------------------|
| **title**              | object          | The title of the movie.                                                          |
| **release_year**       | int64           | The year the movie was released.                                                 |
| **tagline**            | object          | The movie's tagline (promotional phrase).                                        |
| **summary**            | object          | A brief description of the movie's plot.                                         |
| **runtime**            | int64           | The total runtime of the movie in minutes.                                       |
| **letterboxd_rating**  | float64         | The movie's average rating on Letterboxd.                                        |
| **genres**             | object          | A list of genres the movie belongs to (e.g., Drama, Comedy).                     |
| **language**           | object          | The languages the movie was produced in.                                         |
| **countries**          | object          | The countries where the movie was made or released.                              |
| **themes**             | object          | The central themes explored in the movie (e.g., Love, War, Friendship).          |
| **director**           | object          | The director(s) of the movie.                                                   |
| **events**             | object          | Key events or warnings in the movie (e.g., violence, strong language).           |
| **has_warnings**       | bool            | A boolean indicating if the movie contains warnings for sensitive content.       |


In [19]:
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import nltk

import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import plotly.express as px

In [14]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/bru/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
films = pd.read_csv('../data/clean/letterboxd_clean_films.csv')
films.head()

,letterboxd_id,title,release_year,tagline,summary,runtime,letterboxd_rating,genres,language,countries,themes,director,topics,doesthedog_id,events,has_warnings
0,1000001,Barbie,2023,She's everything. He's just Ken.,Barbie and Ken are having the time of their li...,114,3.86,"Comedy, Adventure",English,"UK, USA","['Humanity and the world around us', 'Crude hu...",Greta Gerwig,NaN,381345.0,NaN,False
1,1000002,Parasite,2019,Act like you own the place.,"All unemployed, Ki-taek's family takes peculia...",133,4.56,"Comedy, Thriller, Drama","Korean, English, German, Korean",South Korea,"['Humanity and the world around us', 'Intense ...",Bong Joon-ho,NaN,19408.0,NaN,False
2,1000003,Everything Everywhere All at Once,2022,The universe is so much bigger than you realize.,An aging Chinese immigrant is swept up in an i...,140,4.30,"Science Fiction, Adventure, Comedy, Action","English, Cantonese, Chinese, English",USA,"['Humanity and the world around us', 'Moving r...","Daniel Scheinert, Daniel Kwan",NaN,121671.0,NaN,False
3,1000004,Fight Club,1999,Mischief. Mayhem. Soap.,A ticking-time-bomb insomniac and a slippery s...,139,4.27,Drama,English,"Germany, USA","['Intense violence and sexual transgression', ...",David Fincher,NaN,9593.0,NaN,False
4,1000005,La La Land,2016,Here's to the fools who dream.,"Mia, an aspiring actress, serves lattes to mov...",129,4.09,"Drama, Comedy, Music, Romance",English,"Hong Kong, USA","['Song and dance', 'Humanity and the world aro...",Damien Chazelle,"167,176,222,260,266,269,339,363",12823.0,"flashing lights or images, shower scenes, sad ...",True


In [6]:
cleaned_films = films.copy()
cleaned_films.drop(columns=['topics', 'doesthedog_id', 'letterboxd_id'], inplace=True)

In [7]:
cleaned_films = cleaned_films.dropna()


In [9]:
display(cleaned_films)


,title,release_year,tagline,summary,runtime,letterboxd_rating,genres,language,countries,themes,director,events,has_warnings
4,La La Land,2016,Here's to the fools who dream.,"Mia, an aspiring actress, serves lattes to mov...",129,4.09,"Drama, Comedy, Music, Romance",English,"Hong Kong, USA","['Song and dance', 'Humanity and the world aro...",Damien Chazelle,"flashing lights or images, shower scenes, sad ...",True
11,Whiplash,2014,The road to greatness can take you to the edge.,"Under the direction of a ruthless instructor, ...",107,4.43,"Drama, Music",English,USA,"['Moving relationship stories', 'Student comin...",Damien Chazelle,"finger or toe mutilation, spitting, car crashe...",True
32,Once Upon a Time in Hollywood,2019,"In this town, it can all change… like that","Los Angeles, 1969. TV star Rick Dalton, a stru...",162,3.76,"Drama, Thriller, Comedy","English, English, Italian, Spanish","China, UK, USA","['Humanity and the world around us', 'Fascinat...",Quentin Tarantino,"people being burned alive, spitting, blood or ...",True
39,Glass Onion,2022,"When the game ends, the mystery begins.",World-famous detective Benoit Blanc heads to G...,140,3.45,"Comedy, Crime, Mystery",English,USA,"['Thrillers and murder mysteries', 'Intriguing...",Rian Johnson,"flashing lights or images, car crashes, people...",True
68,Coco,2017,The celebration of a lifetime,Despite his family’s baffling generations-old ...,105,4.12,"Adventure, Animation, Music, Family","English, English, Spanish",USA,"['Moving relationship stories', 'Song and danc...",Lee Unkrich,"parents dying, spitting, ghosts, child abuse, ...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18425,The Triangle,2001,"In the Bermuda Triangle, nothing stays lost fo...",This made-for-TV movie follows a group of frie...,92,2.89,"Thriller, Horror, TV Movie",English,"USA, Canada, Barbados","['Horror, the undead and monster classics', 'T...",Lewis Teague,"kids dying, parents dying, shaving or cutting,...",True
18427,CAT,2022,Drugs. Deceit. Danger.,"Living under an alias, a former police informa...",360,3.50,"Crime, Drama",Hindi,India,"['Crime, drugs and gangsters', 'Intense politi...",Balwinder Singh Janjua,"people being burned alive, flashing lights or ...",True
18429,Wraith,2017,There's Something in My Room,After living in an old mansion for almost 10 y...,99,2.61,"Mystery, Thriller, Horror",English,USA,"['Faith and religion', 'Terrifying, haunted, a...",Michael O. Sajbel,"people being burned alive, spitting, shaky cam...",True
18436,Tulsa,2020,Big changes come in small packages,A desperate marine biker’s life is turned upsi...,120,2.97,"Comedy, Drama",English,USA,"['Faith and religion', 'Moving relationship st...","Gloria Stella, Scott Pryor","kids dying, parents dying, car crashes, people...",True


1. Data Preprocessing

Preprocess the 'tagline' column

In [15]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

cleaned_films['processed_tagline'] = cleaned_films['tagline'].apply(preprocess_text)

In [18]:
# Flatten the list of tokens across all taglines
all_tokens = [word for tokens in cleaned_films['processed_tagline'] for word in tokens]

# Count word frequencies
word_freq = Counter(all_tokens)

# Display the most common words
most_common_words = word_freq.most_common(50)
print('Most common words in taglines:')
print(most_common_words)

Most common words in taglines:
[('one', 249), ('love', 237), ('story', 155), ('never', 154), ('life', 137), ('world', 123), ('time', 111), ('dont', 100), ('evil', 90), ('new', 85), ('every', 84), ('family', 81), ('terror', 78), ('get', 70), ('man', 65), ('go', 64), ('cant', 62), ('true', 62), ('die', 62), ('hell', 62), ('back', 60), ('like', 58), ('fear', 58), ('hes', 58), ('kill', 58), ('woman', 57), ('lives', 56), ('death', 56), ('way', 56), ('theres', 55), ('find', 55), ('would', 55), ('us', 53), ('ever', 53), ('take', 50), ('make', 50), ('first', 47), ('live', 47), ('come', 47), ('nothing', 47), ('know', 46), ('dead', 46), ('human', 44), ('people', 43), ('night', 43), ('home', 42), ('see', 42), ('could', 41), ('everything', 40), ('everyone', 40)]


In [20]:
# Plot the top 10 most common words using Plotly
words, counts = zip(*most_common_words)
fig = px.bar(x=words, y=counts, title='Top 10 Most Common Words in Taglines',
             labels={'x': 'Words', 'y': 'Frequency'}, color=counts)
fig.update_layout(xaxis_tickangle=-45)
fig.show()
